In [1]:
import pyscf
from pyscf import gto, scf, dft
import numpy
import itertools
import pandas
pandas.set_option('display.max_columns', 500)


def run(method, molecule, basis,
        charge=0, ecp={}, spin=0, unit='Angstrom',
        conv_tol=1e-12, conv_tol_grad=1e-8, direct_scf_tol=1e-13,
        init_guess='minao', level_shift=0, max_cycle=100, max_memory=8000,
        xc=None, nlc='', xc_grid=3, nlc_grid=1, small_rho_cutoff=1e-7,
        atomic_radii='BRAGG', becke_scheme='BECKE', prune='NWCHEM',
        radi_method='TREUTLER_AHLRICHS', radii_adjust='TREUTLER',
        algo='DIIS', datapath='', getdict=False, lin_dep_thresh=1e-8,
        prop=False, scf_type=None, stable=False, stable_cyc=3, verbose=0):

    # modify inputs arguments that are strings
    method = method.upper()
    atomic_radii = atomic_radii.upper()
    becke_scheme = becke_scheme.upper()
    if isinstance(prune, str):
        prune = prune.upper()
    radi_method = radi_method.upper()
    if isinstance(radii_adjust, str):
        radii_adjust = radii_adjust.upper()
    algo = algo.upper()

    # create molecule object
    mol = gto.Mole()

    # set mol object attributes
    try:
        gto.Mole(atom=molecule, charge=charge, spin=spin).build()
    except KeyError:
        (mol.atom, charge, spin) = read_molecule(datapath + molecule)
    else:
        mol.atom = molecule
    mol.basis = basis
    mol.charge = charge
    mol.ecp = ecp
    mol.spin = spin
    mol.unit = unit
    mol.verbose = verbose
    mol.build()

    # check method for density functional
    DFT = False
    if method != 'HF':
        try:
            dft.libxc.parse_xc(method)
        except KeyError:
            pass
        else:
            xc = method
            method = 'KS'

    # determine restricted/unrestricted if unspecified
    # automatically sets R if closed-shell and U if open-shell
    if method in ['HF', 'KS', 'DFT'] and scf_type is None:
        if spin == 0:
            scf_type = 'R'
        else:
            scf_type = 'U'

    # create HF/KS object
    if method in ['RHF', 'ROHF'] or (method == 'HF' and scf_type in ['R', 'RO']):
        mf = scf.RHF(mol)
        scf_type = 'R'
    elif method == 'UHF' or (method == 'HF' and scf_type == 'U'):
        mf = scf.UHF(mol)
        scf_type = 'U'
    elif method in ['RKS', 'ROKS', 'RDFT', 'RODFT'] or (method in ['KS', 'DFT'] and scf_type in ['R', 'RO']):
        mf = scf.RKS(mol)
        scf_type = 'R'
        DFT = True
    elif method in ['UKS', 'UDFT'] or (method in ['KS', 'DFT'] and scf_type == 'U'):
        mf = scf.UKS(mol)
        scf_type = 'U'
        DFT = True
    else:
        print "CRASH 1"

    # set HF attributes
    mf.conv_check = False
    mf.conv_tol = conv_tol
    mf.conv_tol_grad = conv_tol_grad
    mf.direct_scf_tol = direct_scf_tol
    mf.init_guess = init_guess
    mf.level_shift = level_shift
    mf.max_cycle = max_cycle
    mf.max_memory = max_memory
    mf.verbose = verbose

    # set KS attributes
    if DFT:

        mf.xc = xc
        if mf.xc is None:
            print "CRASH 2"
        mf.nlc = nlc

        if isinstance(xc_grid, int):
            mf.grids.level = xc_grid
        elif isinstance(xc_grid, tuple) or isinstance(xc_grid, dict):
            mf.grids.atom_grid = xc_grid
        else:
            print "CRASH 3"

        if isinstance(nlc_grid, int):
            mf.nlcgrids.level = nlc_grid
        elif isinstance(nlc_grid, tuple) or isinstance(nlc_grid, dict):
            mf.nlcgrids.atom_grid = nlc_grid
        else:
            print "CRASH 4"

        if atomic_radii == 'BRAGG':
            mf.grids.atomic_radii = dft.radi.BRAGG_RADII
            mf.nlcgrids.atomic_radii = dft.radi.BRAGG_RADII
        elif atomic_radii == 'COVALENT':
            mf.grids.atomic_radii = dft.radi.COVALENT_RADII
            mf.nlcgrids.atomic_radii = dft.radi.COVALENT_RADII
        else:
            print "CRASH 5"

        if becke_scheme == 'BECKE':
            mf.grids.becke_scheme = dft.gen_grid.original_becke
            mf.nlcgrids.becke_scheme = dft.gen_grid.original_becke
        elif becke_scheme == 'STRATMANN':
            mf.grids.becke_scheme = dft.gen_grid.stratmann
            mf.nlcgrids.becke_scheme = dft.gen_grid.stratmann
        else:
            print "CRASH 6"

        if prune == 'NWCHEM':
            mf.grids.prune = dft.gen_grid.nwchem_prune
            mf.nlcgrids.prune = dft.gen_grid.nwchem_prune
        elif prune == 'SG1':
            mf.grids.prune = dft.gen_grid.sg1_prune
            mf.nlcgrids.prune = dft.gen_grid.sg1_prune
        elif prune == 'TREUTLER':
            mf.grids.prune = dft.gen_grid.treutler_prune
            mf.nlcgrids.prune = dft.gen_grid.treutler_prune
        elif prune == 'NONE' or prune is None:
            mf.grids.prune = None
            mf.nlcgrids.prune = None
        else:
            print "CRASH 7"

        if radi_method in ['TREUTLER_AHLRICHS', 'TREUTLER', 'AHLRICHS']:
            mf.grids.radi_method = dft.radi.treutler_ahlrichs
            mf.nlcgrids.radi_method = dft.radi.treutler_ahlrichs
        elif radi_method == 'DELLEY':
            mf.grids.radi_method = dft.radi.delley
            mf.nlcgrids.radi_method = dft.radi.delley
        elif radi_method in ['MURA_KNOWLES', 'MURA', 'KNOWLES']:
            mf.grids.radi_method = dft.radi.mura_knowles
            mf.nlcgrids.radi_method = dft.radi.mura_knowles
        elif radi_method in ['GAUSS_CHEBYSHEV', 'GAUSS', 'CHEBYSHEV']:
            mf.grids.radi_method = dft.radi.gauss_chebyshev
            mf.nlcgrids.radi_method = dft.radi.gauss_chebyshev
        else:
            print "CRASH 8"

        if radii_adjust == 'TREUTLER':
            mf.grids.radii_adjust = dft.radi.treutler_atomic_radii_adjust
            mf.nlcgrids.radii_adjust = dft.radi.treutler_atomic_radii_adjust
        elif radii_adjust == 'BECKE':
            mf.grids.radii_adjust = dft.radi.becke_atomic_radii_adjust
            mf.nlcgrids.radii_adjust = dft.radi.becke_atomic_radii_adjust
        elif radii_adjust == 'NONE' or radii_adjust is None:
            mf.grids.radii_adjust = None
            mf.nlcgrids.radii_adjust = None
        else:
            print "CRASH 9"

        mf.small_rho_cutoff = small_rho_cutoff

    # select optimizer
    if algo == 'DIIS':
        mf.diis = True
    elif algo == 'ADIIS':
        mf.diis = scf.diis.ADIIS()
    elif algo == 'EDIIS':
        mf.diis = scf.diis.EDIIS()
    elif algo == 'NEWTON':
        mf = mf.newton()
    else:
        print "CRASH 10"

    # run SCF
    mf.kernel()

    # stability analysis (optional)
    if stable:
        print 'Initial SCF energy: ', mf.e_tot
        print 'Performing Stability Analysis (up to ', stable_cyc, 'iterations)'
        for i in range(stable_cyc):
            new_mo_coeff = mf.stability(internal=True, external=False)[0]
            if numpy.linalg.norm(numpy.array(new_mo_coeff) - numpy.array(mf.mo_coeff)) < 10**-14:
                print "Stable!"
                break
            else:
                print "Unstable!"
                if scf_type == 'U':
                    n_alpha = numpy.count_nonzero(mf.mo_occ[0])
                    n_beta = numpy.count_nonzero(mf.mo_occ[1])
                    P_alpha = numpy.dot(new_mo_coeff[0][:, :n_alpha], new_mo_coeff[0].T[:n_alpha])
                    P_beta = numpy.dot(new_mo_coeff[1][:, :n_beta], new_mo_coeff[1].T[:n_beta])
                    mf.kernel(dm0=(P_alpha, P_beta))
                elif scf_type in ['R', 'RO']:
                    n_alpha = numpy.count_nonzero(mf.mo_occ)
                    P_alpha = 2*numpy.dot(new_mo_coeff[:, :n_alpha], new_mo_coeff.T[:n_alpha])
                    mf.kernel(dm0=(P_alpha))
                else:
                    print "CRASH 11"
                print 'Updated SCF energy: ', mf.e_tot

    # properties
    if prop:

        # dipole moment
        mf.dip_moment()

        # S^2
        if scf_type == 'U':
            (ssq, mult) = mf.spin_square()
            print '(S^2, 2S+1):', (ssq, mult)
        elif scf_type in ['R', 'RO']:
            S = float(mol.spin)/2.
            (ssq, mult) = (S*(S+1.), 2.*S+1.)
            print '(S^2, 2S+1):', (ssq, mult)
        else:
            print "CRASH 12"

        # population analysis
        print "Mulliken population analysis"
        for ia in range(mol.natm):
            symb = mol.atom_symbol(ia)
            print symb, ':', mf.mulliken_meta(verbose=verbose)[1][ia]

        print "Mulliken population analysis, based on meta-Lowdin AOs"
        for ia in range(mol.natm):
            symb = mol.atom_symbol(ia)
            print symb, ':', mf.mulliken_pop(verbose=verbose)[1][ia]

    # return either just an energy or (energy,dict)
    if getdict:
        mydict = dict()
        mydict.update(mol.__dict__)
        mydict.update(mf.__dict__)
        if DFT:
            mydict.update(mf.grids.__dict__)
        return mf.e_tot, mydict
    else:
        return mf.e_tot


def run_batch(method, molecule, basis, database=pandas.DataFrame(), **kwargs):
    mand_args = ['method', 'molecule', 'basis']
    num_mand_args = len(mand_args)
    if not isinstance(method, list):
        method = [method]
    if not isinstance(molecule, list):
        molecule = [molecule]
    if not isinstance(basis, list):
        basis = [basis]
    for i in itertools.product(method, molecule, basis, list(parse_kwargs(**kwargs))):
        mydict = dict()
        mydict.update(i[num_mand_args:][0])
        (en, rundict) = run(*i[:num_mand_args], getdict=True, **i[num_mand_args:][0])
        mydict.update(rundict)
        for num, val in enumerate(mand_args):
            if val == 'molecule':
                try:
                    read_molecule(mydict['datapath'] + i[num])
                except IOError:
                    mydict.update({val: XYZtoMF(i[num])})
                else:
                    mydict.update({val: i[num]})
            else:
                mydict.update({val: i[num]})
        database = database.append(mydict, ignore_index=True)

    return database


def parse_kwargs(**kwargs):
    items = kwargs.items()
    keys = [key for key, value in items]
    sets = [value for key, value in items]
    for index, item in enumerate(sets):
        if not isinstance(item, list):
            sets[index] = [sets[index]]
    for values in itertools.product(*sets):
        yield dict(zip(keys, values))


def read_molecule(path):

    charge = spin = 0
    with open(path, 'r') as myfile:
        output = myfile.read()
        output = output.lstrip()
        output = output.rstrip()
        output = output.split('\n')

    try:
        int(output[0])
    except ValueError:
        try:
            charge = int(output[0].split(' ')[0])
            spin = int(output[0].split(' ')[1]) - 1
        except ValueError:
            molecule = output
        else:
            molecule = '\n'.join(output[1:])
    else:
        if int(output[0]) == len(output) - 2:
            molecule = '\n'.join(output[2:])
            try:
                charge = int(output[1].split(' ')[0])
                spin = int(output[1].split(' ')[1])-1
            except ValueError:
                pass
        else:
            print "THIS IS NOT A VALID XYZ FILE"

    return (molecule, charge, spin)


def XYZtoMF(inp):

    return filter(lambda x: x.isalpha(), inp)

# Introduction: Welcome to the QuSim User Guide

The QuSim user interface is the easiest way to perform computational chemistry calculations using PySCF. The `run` function accepts three mandatory, ordered inputs (as well as a variety of optional ones):

* method
* geometry
* basis

This guide will cover the basics of the QuSim user interface and demonstrate the computation of various types of chemical interactions such as atomization energies, bond dissociation energies, binding energies, etc. Furthermore, it will highlight the sensitivity of certain chemical interactions and methods to both the basis set as well as the integration grid (which pertains specifically to DFT).

To begin, we will calculate the energy of H$_2$ with a bond length of 0.74 Å. First, it is necessary to define the geometry:

In [2]:
h2="""
H 0.0 0.0 0.0
H 0.0 0.0 0.74"""

For this specific example, we will use Hartree–Fock (HF) and the double-zeta cc-pVDZ basis set:

In [3]:
run('HF', h2, 'cc-pVDZ')

-1.1287000935564406

This should return an energy of -1.1287000935564406 Hartree.

The `run` function has a variety of useful, optional features in addition to the three mandatory ones. For example, one can set the charge and spin of a molecule as follows:

In [4]:
run('HF', h2, 'cc-pVDZ', charge=1, spin=1)

-0.5652012007354672

The calculation above computes the energy of H$_2$$^+$, which has a single unpaired electron (hence `spin=1`, where spin is equal to 2S).

By default, `run` will perform a restricted calculation if the molecule is closed-shell (no unpaired electrons or `spin=0`) and an unrestricted calculation if the molecule is open-shell (`spin>0`). This default can be overwritten in several ways. Either the method can be explicitly stated (i.e., `RHF`, `ROHF`, or `UHF`), or the `scf_type` input can be set explicitly (`R`, `RO`, or `U`).

As an example, we will compute the energy of an open-shell molecule, namely, the hydroxyl radical, OH•.

In [5]:
oh="""
O 0.0000000000 0.0000000000 0.0000000000
H 0.0000000000 0.0000000000 0.9706601900"""

The following demonstrates the various ways of performing unrestricted vs. restricted calculations. The first three lines should produce identical results corresponding to an unrestricted calculation of OH•, while the last four lines should produce identical results corresponding to a restricted calculation of OH•.

In [6]:
# unrestricted
print 'HF: ', run('HF', oh, 'cc-pVDZ', spin=1)
print 'UHF: ', run('UHF', oh, 'cc-pVDZ', spin=1)
print 'HF/U: ', run('HF', oh, 'cc-pVDZ', spin=1, scf_type='U')

print ''

# restricted
print 'RHF: ', run('RHF', oh, 'cc-pVDZ', spin=1)
print 'ROHF: ', run('ROHF', oh, 'cc-pVDZ', spin=1)
print 'HF/R: ', run('HF', oh, 'cc-pVDZ', spin=1, scf_type='R')
print 'HF/RO: ', run('HF', oh, 'cc-pVDZ', spin=1, scf_type='RO')

HF:  -75.3938226865
UHF:  -75.3938226865
HF/U:  -75.3938226865

RHF:  -75.3899856282
ROHF:  -75.3899856282
HF/R:  -75.3899856282
HF/RO:  -75.3899856282


Naturally, the first three will all return the lower energy corresponding to a UHF calculation, while the rest will return a higher energy corresponding to an ROHF calculation.

Another useful feature that can be turned on is `prop`. When `prop=True`, a variety of molecular properties (dipole moment, S$^2$ and 2S+1 values, as well as two different population analysis results) are computed after the SCF converges. For example:

In [7]:
run('UHF', oh, 'cc-pVDZ', spin=1, prop=True)

Dipole moment(X, Y, Z, A.U.):  0.00000,  0.00000,  1.80400
(S^2, 2S+1): (0.75461173279801885, 2.0046064280032816)
Mulliken population analysis
O : -0.323214025599
H : 0.323214025599
Mulliken population analysis, based on meta-Lowdin AOs
O : -0.184995587217
H : 0.184995587217


-75.393822686483375

When dealing with radicals, multireference systems, or systems that can break spin symmetry, it is possible to land on an SCF solution that is unstable. For these notorious cases, the `stable=True` setting will attempt to resolve the instability 3 times (a setting that can be modified by `stable_cyc`). As an example, consider the singlet C$_2$ molecule. First, we will perform an RHF calculation without stability analysis:

In [8]:
c2="""
C 0.0 0.0 0.0
C 0.0 0.0 1.24"""

In [9]:
run('RHF', c2, 'cc-pVDZ')

-75.386817113966686

This returns an energy of -75.386817113966686 Hartree. Running the same calculation with stability analysis confirms that the previous energy was a saddle point, and returns a lower restricted energy:

In [10]:
run('RHF', c2, 'cc-pVDZ', stable=True)

Initial SCF energy:  -75.386817114
Performing Stability Analysis (up to  3 iterations)
Unstable!
Updated SCF energy:  -75.4159592748
Stable!


-75.41595927475602

The stable energy of -75.41595927475602 Hartree is lower than the unstable one by 18.29 kcal/mol! Thus, if one wanted to compute the bond dissociation energy of C$_2$ (C$_2$ - 2C), landing on the unstable solution would result in an error of nearly 20 kcal/mol.

Since we are dealing with singlet C$_2$ in this example, one could expect an RHF calculation to be sufficient. However, the following UHF calculation demonstrates that C$_2$ can break spin symmetry:

In [11]:
c2="""
C 0.0 0.0 0.0
C 0.0 0.0 1.24"""

run('UHF', c2, 'cc-pVDZ', stable=True, prop=True)

Initial SCF energy:  -75.386817114
Performing Stability Analysis (up to  3 iterations)
Unstable!
Updated SCF energy:  -75.5053140149
Stable!
Dipole moment(X, Y, Z, A.U.):  0.00000, -0.00000,  0.00000
(S^2, 2S+1): (1.6577115386994441, 2.7623986234426372)
Mulliken population analysis
C : 1.15574838588e-09
C : -1.15574483317e-09
Mulliken population analysis, based on meta-Lowdin AOs
C : 6.80303813283e-10
C : -6.80300260569e-10


-75.505314014915271

Initially, the UHF calculation lands on the same unstable solution as RHF: -75.386817114 Hartree. However, once the instability has been resolved, the stable UHF energy of -75.5053140149 Hartree is 74.36 kcal/mol lower than the unstable solution, and 56.07 kcal/mol lower than the stable RHF solution. While the stable RHF solution has an S$^2$ value of 0, the stable UHF solution has an S$^2$ value of 1.66.

Another useful feature is the ability to read molecule files from disk. The parser can automatically detect any of the three file formats contained in the geom folder (XYZ, MOL, QC). These formats are explained [here](#atom). It also takes an input variable called `datapath` which is the absolute or relative path to the directory where the files are contained. For example, the geom folder contains:

* water_dimer.mol
* water_dimer.qc
* water_dimer.xyz

The energies for the molecules contained in these three files can be computed very easily:

In [12]:
print run('HF', 'water_dimer.mol', 'cc-pVDZ', datapath='geom/')
print run('HF', 'water_dimer.qc', 'cc-pVDZ', datapath='geom/')
print run('HF', 'water_dimer.xyz', 'cc-pVDZ', datapath='geom/')

-152.06253625
-152.06253625
-152.06253625


This tutorial is also equipped with all of the molecules from the MGCDB84 database, found in this [DFT review paper](https://www.tandfonline.com/doi/full/10.1080/00268976.2017.1333644). These molecules can be found in the data folder. Running any of these molecules (ethyne for example) is also straightforward:

In [13]:
print run('HF', '126_c2h2_W4-11.xyz', 'cc-pVDZ', datapath='data/')

-76.8255572993


Because the QC file format contains the charge and multiplicity of the molecule, it is not necessary to specify the spin (for example) when reading in the hydroxyl radical file (222_oh_W4-11.xyz) from the data directory:

In [14]:
print run('HF', '222_oh_W4-11.xyz', 'cc-pVDZ', datapath='data/')

-75.3938226865


Another feature of the QuSim user interface is the ability to run batch jobs with the `run_batch` function. This function is essentially a wrapper for `run` and takes the same inputs. The first three inputs (method, molecule, basis) are mandatory, and can be lists. The optional arguments can also be lists.

In [2]:
filenames=['11_Ar-Ar_dim_NC15.xyz', '11_Ar-Ar_monA_NC15.xyz', '11_Ar-Ar_monB_NC15.xyz']

mydbase=run_batch(['UHF', 'RHF'], filenames, ['cc-pVDZ', 'def2-SVP'], datapath='data/')

This will return a pandas DataFrame that contains all of the data embedded in the molecule and mean-field objects. A truncated version of this DataFrame can be accessed via:

In [3]:
mydbase[['molecule', 'method', 'basis', 'charge', 'spin', 'e_tot', 'converged']]

,molecule,method,basis,charge,spin,e_tot,converged
0,11_Ar-Ar_dim_NC15.xyz,UHF,cc-pVDZ,0.0,0.0,-1053.599476,1.0
1,11_Ar-Ar_dim_NC15.xyz,UHF,def2-SVP,0.0,0.0,-1053.246842,1.0
2,11_Ar-Ar_monA_NC15.xyz,UHF,cc-pVDZ,0.0,0.0,-526.799865,1.0
3,11_Ar-Ar_monA_NC15.xyz,UHF,def2-SVP,0.0,0.0,-526.623385,1.0
4,11_Ar-Ar_monB_NC15.xyz,UHF,cc-pVDZ,0.0,0.0,-526.799865,1.0
5,11_Ar-Ar_monB_NC15.xyz,UHF,def2-SVP,0.0,0.0,-526.623385,1.0
6,11_Ar-Ar_dim_NC15.xyz,RHF,cc-pVDZ,0.0,0.0,-1053.599476,1.0
7,11_Ar-Ar_dim_NC15.xyz,RHF,def2-SVP,0.0,0.0,-1053.246842,1.0
8,11_Ar-Ar_monA_NC15.xyz,RHF,cc-pVDZ,0.0,0.0,-526.799865,1.0
9,11_Ar-Ar_monA_NC15.xyz,RHF,def2-SVP,0.0,0.0,-526.623385,1.0


You can keep adding to the database if you specify the name of the database as an argument to run_batch:

In [17]:
mydbase=run_batch(['UHF', 'RHF'], filenames, ['def2-SVPD', '6-31G'], database=mydbase, datapath='data/')

In [18]:
mydbase[['molecule', 'method', 'basis', 'charge', 'spin', 'e_tot', 'converged']]

,molecule,method,basis,charge,spin,e_tot,converged
0,11_Ar-Ar_dim_NC15.xyz,UHF,cc-pVDZ,0.0,0.0,-1053.599476,1.0
1,11_Ar-Ar_dim_NC15.xyz,UHF,def2-SVP,0.0,0.0,-1053.246842,1.0
2,11_Ar-Ar_monA_NC15.xyz,UHF,cc-pVDZ,0.0,0.0,-526.799865,1.0
3,11_Ar-Ar_monA_NC15.xyz,UHF,def2-SVP,0.0,0.0,-526.623385,1.0
4,11_Ar-Ar_monB_NC15.xyz,UHF,cc-pVDZ,0.0,0.0,-526.799865,1.0
5,11_Ar-Ar_monB_NC15.xyz,UHF,def2-SVP,0.0,0.0,-526.623385,1.0
6,11_Ar-Ar_dim_NC15.xyz,RHF,cc-pVDZ,0.0,0.0,-1053.599476,1.0
7,11_Ar-Ar_dim_NC15.xyz,RHF,def2-SVP,0.0,0.0,-1053.246842,1.0
8,11_Ar-Ar_monA_NC15.xyz,RHF,cc-pVDZ,0.0,0.0,-526.799865,1.0
9,11_Ar-Ar_monA_NC15.xyz,RHF,def2-SVP,0.0,0.0,-526.623385,1.0


The full pandas DataFrame contains a large amount of data:

In [19]:
mydbase.head(2)

,_atm,_atom,_bas,_basis,_built,_chkfile,_ecp,_ecpbas,_env,_eri,_keys,_nelectron,_t0,_w0,atom,basis,callback,cart,charge,chkfile,conv_check,conv_tol,conv_tol_grad,converged,datapath,diis,direct_scf_tol,e_tot,ecp,groupname,init_guess,irrep_id,irrep_name,level_shift,max_cycle,max_memory,method,mo_coeff,mo_energy,mo_occ,mol,molecule,nelec,nucmod,opt,output,spin,stdout,symm_orb,symmetry,symmetry_subgroup,topgroup,unit,verbose
0,"[[18, 20, 1, 23, 0, 0], [18, 24, 1, 27, 0, 0]]","[(Ar, [3.54323648356, 0.0, 0.0]), (Ar, [-3.543...","[[0, 0, 11, 3, 0, 28, 39, 0], [0, 0, 1, 1, 0, ...","{u'Ar': [[0, [145700.0, 0.0002367, -6.7491e-05...",1.0,"<open file '<fdopen>', mode 'w+b' at 0x7f40e5c...",[],[],"[137.03599968, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[10.9203114287, -1.31592562125, 0.263519480103...","{opt, direct_scf, verbose, stdout, level_shift...",None,5.259812,1.523684e+09,Ar 1.8750000000 0.0000000000 0.0000000000\nAr ...,cc-pVDZ,None,0.0,0.0,/home/narbe/Tutorial_PySCF/tmp7YJ7oE,0.0,1.000000e-12,1.000000e-08,1.0,data/,1.0,1.000000e-13,-1053.599476,{},C1,minao,None,None,0.0,100.0,8000.0,UHF,"([[0.707120975302, 0.707120352269, -5.71489765...","([-118.6061178, -118.606117699, -12.3176581742...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",<pyscf.gto.mole.Mole object at 0x7f40e5bc1ed0>,11_Ar-Ar_dim_NC15.xyz,None,{},None,None,0.0,<ipykernel.iostream.OutStream object at 0x7f41...,None,0.0,None,C1,Angstrom,0.0
1,"[[18, 20, 1, 23, 0, 0], [18, 24, 1, 27, 0, 0]]","[(Ar, [3.54323648356, 0.0, 0.0]), (Ar, [-3.543...","[[0, 0, 5, 1, 0, 28, 33, 0], [0, 0, 3, 1, 0, 3...","{u'Ar': [[0, [11797.119765, 0.0020214479973], ...",1.0,"<open file '<fdopen>', mode 'w+b' at 0x7f40e5c...",[],[],"[137.03599968, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[11.0233591869, 2.02986621851, 0.460838722679,...","{opt, direct_scf, verbose, stdout, level_shift...",None,5.454399,1.523684e+09,Ar 1.8750000000 0.0000000000 0.0000000000\nAr ...,def2-SVP,None,0.0,0.0,/home/narbe/Tutorial_PySCF/tmp0aL37x,0.0,1.000000e-12,1.000000e-08,1.0,data/,1.0,1.000000e-13,-1053.246842,{},C1,minao,None,None,0.0,100.0,8000.0,UHF,"([[0.702752480149, 0.702755043495, -0.22449745...","([-118.549384229, -118.549367281, -12.30248187...","[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...",<pyscf.gto.mole.Mole object at 0x7f411441e410>,11_Ar-Ar_dim_NC15.xyz,None,{},None,None,0.0,<ipykernel.iostream.OutStream object at 0x7f41...,None,0.0,None,C1,Angstrom,0.0


---

ENDS HERE FOR NOW!

---

In [5]:
mol_list=['169_f2_W4-11.xyz',"""F"""]
mol_int=[1,-2]
spin_int=[0,1]
basis_set_list=['6-31G','cc-pVDZ','cc-pVTZ','cc-pVQZ','cc-pV5Z']

for j in range(len(basis_set_list)):
    en=0.
    for i in range(len(mol_list)):
        en+=mol_int[i]*run('0.2*HF+0.08*LDA+0.72*B88,0.19*VWN_RPA+0.81*LYP',mol_list[i],basis_set_list[j],spin=spin_int[i],stable=False,datapath='/home/narbe/Tutorial_PySCF/data/',scf_type='U')
    print basis_set_list[j], ':', en*627.5095

6-31G : -30.3460907145
cc-pVDZ : -37.7285547963
cc-pVTZ : -37.9614307675
cc-pVQZ : -37.4238020232
cc-pV5Z : -37.0073200144


In [10]:
#mol.multiplicity=1 for closed shell
#gaussian basis paerser in basis/parse_gaussian.py

#TO-DO LIST

#allow mol.atom to take something like @H or !H -- something to indicate Ghost easier (X-H updates)
#mol.cart needs to be set based on the basis set (mostly for Pople -- do we care?)
#be able to read mol.atom from any file, and the file can either be just XYZ/Zmat, coordinates, Q-Chem like, etc.

#D3 has to be implemented, and VV10 sped up, and maybe D2 D3(BJ) D4 and all that new stuff

#what about fragment monomoners?
#be able to take input of Gaussian94 basis/ecp
#be able to read basis from file on disk both formats
#maybe for mol.cart instead of 0 and 1, do 'spherical' and 'Cartesian'

#there should be a verbose level for Mole that doesn't print basis set coeffs and stuff
#shouldn't the extra cycle for SCF (conv_check) only be initiated IF there is a level shift?
#should we be using the norm of the orbital gradient for convergence or the rms
#printing HOMO/LUMO on each cycle is unnecessary or at least we should be able to turn it off
#we should print like orbital energies at end, split into occ and virt

#what exactly is the minao guess? will it be expensive for large systems?
#mp.frozen and cc.frozen should have a default!! like if true, it should just do something obvious
#pt2 kernel should not return t2?!
#cc kernel shouldt return t2?!
#it's not clear what the default SCF conv_tol is? 1e-9 or 1e-10? it is 1e-9
#nalpha and nbeta for UHF?

#if atom_grid is specified, then don't print grids.level, that is confusing
#some properties should be computed for free and by default -- atomic charges, S^2, multipole moments, mulliken population
    #group all properties into def analyze and just make it default run in kernel
#lin dep should be automatically removed!!!
#stability_analysis easier
#opitmizer options: diis, ediis, adiis, newton, gdm
#breakdown of energy printed at end (nuclear rep./1e/2e/DFT/disp/PCM/EFP/total)
#for DFT, it should allow 'M06-L' not 'M06_L,M06_L' need to hard-code the XC
    #for DFT mf.xc=',' in addition to 'HF' maybe we should have 'SR-HF' and 'LR-HF'
#allow setting of b and C for VV10
#give RHF/ROHF a spin square function just to print it out so ppl dont have to compute in their head
#spin per atom for UHF calc?

#PSI4 output info:
#rotational constants?
#charge/mult/electron alph beta
#dipole in different units, both nuclear/electron/full

#NWChem - VWN1RPA
#PySCF - VWN5
#TURBOMOLE - VWN5
#GAMESS - VWN5
#MOLPRO - VWN5
#ORCA - VWN5
#PSI4 - VWN5
#Q-Chem - VWN1RPA
#Gaussian - VWN3

#generate file that run function psueodcode
#mf.RKS()
#...
#mf=mf.newton()

#scf_guess read impement

#        xc=None, nlc='', xc_grid=(99,590), nlc_grid=(50,194), small_rho_cutoff=1e-7,
#        atomic_radii='BRAGG', becke_scheme='BECKE', prune=None, radi_method='GAUSS_CHEBYSHEV', radii_adjust=None,

In [2]:
testdbase=run_batch(['PBE123','PBE,','PBE,PBE'],'h2o_SW49.xyz','cc-pVDZ',datapath='data/')

In [3]:
testdbase[['molecule', 'method', 'basis', 'charge', 'spin', 'e_tot', 'converged']]

,molecule,method,basis,charge,spin,e_tot,converged
0,h2o_SW49.xyz,PBE123,cc-pVDZ,0.0,0.0,-76.333679,1.0
1,h2o_SW49.xyz,"PBE,",cc-pVDZ,0.0,0.0,-76.003041,1.0
2,h2o_SW49.xyz,"PBE,PBE",cc-pVDZ,0.0,0.0,-76.333679,1.0
